# PyTorch 101

Create a conda environment:

`conda create -n pytorch101 pip conda python==3.6.8`

Install PyTorch: https://pytorch.org/get-started/locally/
Install GraphViz: https://www.graphviz.org/download/

Install additionak packages:

`conda install scikit-learn==0.21.3 matplotlib==3.1.1 jupyter -c anaconda`

`pip install torchviz`

## Introduction

**PyTorch** is the **fastest growing** Deep Learning framework and it is also used by **Fast.ai** in its MOOC, [Deep Learning for Coders](https://course.fast.ai/) and its [library](https://docs.fast.ai/).

PyTorch is also very *pythonic*, meaning, it feels more natural to use it if you already are a Python developer.

Besides, using PyTorch may even improve your health, according to [Andrej Karpathy](https://twitter.com/karpathy/status/868178954032513024) :-)


## Motivation

There are *many many* PyTorch tutorials around and its documentation is quite complete and extensive. So, **why** should you keep reading this step-by-step tutorial?

Well, even though one can find information on pretty much anything PyTorch can do, I missed having a **structured, incremental and from first principles** approach to it.

In this tutorial, I will guide you through the *main reasons* why PyTorch makes it much **easier** and more **intuitive** to build a Deep Learning model in Python — **autograd, dynamic computation graph, model classes** and more — and I will also show you **how to avoid some common pitfalls and errors** along the way.

## A Simple Regression Problem

Most tutorials start with some nice and pretty *image classification problem* to illustrate how to use PyTorch. It may seem cool, but I believe it **distracts** you from the **main goal: how PyTorch works**?

For this reason, in this tutorial, I will stick with a **simple** and **familiar** problem: a **linear regression with a single feature x**! It doesn’t get much simpler than that…

$$
\large y = a + b x + \epsilon
$$

### Data Generation

Let’s start **generating** some synthetic data: we start with a vector of 100 points for our **feature x** and create our **labels** using **a = 1, b = 2** and some Gaussian noise.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
true_a = 1
true_b = 2
N = 100

# Data Generation
np.random.seed(42)
x = np.random.rand(N, 1)
y = true_a + true_b * x + .1 * np.random.randn(N, 1)

### Train / Validation Split

Next, let’s **split** our synthetic data into **train** and **validation** sets, shuffling the array of indices and using the first 80 shuffled points for training.

In [ ]:
# Shuffles the indices
idx = np.arange(N)
np.random.shuffle(idx)

# Uses first 80 random indices for train
train_idx = idx[:int(N*.8)]
# Uses the remaining indices for validation
val_idx = idx[int(N*.8):]

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].scatter(x_train, y_train)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[0].set_ylim([1, 3])
ax[0].set_title('Generated Data - Train')
ax[1].scatter(x_val, y_val, c='r')
ax[1].set_xlabel('x')
ax[1].set_ylabel('y')
ax[1].set_ylim([1, 3])
ax[1].set_title('Generated Data - Validation')

We **know** that a = 1 and b = 2, but now let’s see how close we can get to the true values by using **gradient descent** and the 80 points in the **training set**…

## Gradient Descent

It goes beyond the scope of this tutorial to fully explain how gradient descent works, but I'll cover the **five basic steps** you'd need to go through to compute it.

### Step 0: Initialization

Technically, this step is not part of gradient descent, but it is an important step nonetheless.

For training a model, you need to **randomly initialize the parameters/weights** (we have only two, **a** and **b**).

Make sure to *always initialize your random seed* to ensure **reproducibility** of your results. As usual, the random seed is [42](https://en.wikipedia.org/wiki/Phrases_from_The_Hitchhiker%27s_Guide_to_the_Galaxy#Answer_to_the_Ultimate_Question_of_Life,_the_Universe,_and_Everything_(42)), the *least random* of all random seeds one could possibly choose :-)

In [ ]:
np.random.seed(42)
a = np.random.randn(1)
b = np.random.randn(1)

print(a, b)

### Step 1: Compute Model's Predictions

This is the **forward pass** - it simply *computes the model's predictions using the current values of the parameters/weights*. At the very beginning, we will be producing really bad predictions, as we started with random values from Step 0.

In [ ]:
# Computes our model's predicted output
yhat = a + b * x_train

### Step 2: Compute Loss

There is a subtle but fundamental difference between **error** and **loss**. 

The **error** is the difference between **actual** and **predicted** computed for a single data point.

The **loss**, on the other hand, is some sort of **aggregation of errors for a set of data points**.

For a regression problem, the **loss** is given by the **Mean Square Error (MSE)**, that is, the average of all squared differences between **actual values** (y) and **predictions** (a + bx).

---

*It is worth mentioning that, if we use **all points** in the training set (N) to compute the loss, we are performing a **batch** gradient descent. If we were to use a **single point** at each time, it would be a **stochastic** gradient descent. Anything else (n) **in-between 1 and N** characterizes a **mini-batch** gradient descent.*

---


**Loss**:

$$
\large MSE = \frac{1}{N} \sum_{i=1}^N{(y_i - \hat{y_i})}^2
$$

$$
\large MSE = \frac{1}{N} \sum_{i=1}^N{(y_i - a - b x_i)}^2
$$


In [ ]:
# How wrong is our model? That's the error! 
error = (y_train - yhat)

# It is a regression, so it computes mean squared error (MSE)
loss = (error ** 2).mean()

print(loss)

### Step 3: Compute the Gradients

A **gradient** is a **partial derivative** — *why partial*? Because one computes it with respect to (w.r.t.) a **single parameter**. We have two parameters, **a** and **b**, so we must compute two partial derivatives.

A **derivative** tells you *how much* **a given quantity changes** when you *slightly* vary some **other quantity**. In our case, how much does our **MSE** **loss** change when we vary **each one of our two parameters**?

The *right-most* part of the equations below is what you usually see in implementations of gradient descent for a simple linear regression. In the **intermediate step**, I show you **all elements** that pop-up from the application of the [chain rule](https://en.wikipedia.org/wiki/Chain_rule), so you know how the final expression came to be.

**Gradients**:

$$
\large \frac{\partial{MSE}}{\partial{a}} = \frac{\partial{MSE}}{\partial{\hat{y_i}}} \cdot \frac{\partial{\hat{y_i}}}{\partial{a}} = \frac{1}{N} \sum_{i=1}^N{2(y_i - a - b x_i) \cdot (-1)} = -2 \frac{1}{N} \sum_{i=1}^N{(y_i - \hat{y_i})}
$$ 

$$
\large \frac{\partial{MSE}}{\partial{b}} = \frac{\partial{MSE}}{\partial{\hat{y_i}}} \cdot \frac{\partial{\hat{y_i}}}{\partial{b}} = \frac{1}{N} \sum_{i=1}^N{2(y_i - a - b x_i) \cdot (-x_i)} = -2 \frac{1}{N} \sum_{i=1}^N{x_i (y_i - \hat{y_i})}
$$


In [ ]:
# Computes gradients for both "a" and "b" parameters
a_grad = -2 * error.mean()
b_grad = -2 * (x_train * error).mean()
print(a_grad, b_grad)

### Step 4: Update the Parameters

In the final step, we **use the gradients to update** the parameters. Since we are trying to **minimize** our **losses**, we **reverse the sign** of the gradient for the update.

There is still another parameter to consider: the **learning rate**, denoted by the *Greek letter* **eta** (that looks like the letter **n**), which is the **multiplicative factor** that we need to apply to the gradient for the parameter update.

**Parameters**:

$$
\large a = a - \eta \frac{\partial{MSE}}{\partial{a}}
$$

$$
\large b = b - \eta \frac{\partial{MSE}}{\partial{b}}
$$

How to choose a learning rate? That is a topic on its own and beyond the scope of this tutorial as well.

Let's start with a value of **0.1** (which is a relatively *big value*, as far as learning rates are concerned!).

In [ ]:
# Sets learning rate
lr = 1e-1

print(a, b)
# Updates parameters using gradients and the learning rate
a = a - lr * a_grad
b = b - lr * b_grad
print(a, b)

### Step 5: Rinse and Repeat!

Now we use the **updated parameters** to go back to **Step 1** and restart the process.

---

*An **epoch is complete whenever every point has been already used for computing the loss**. For **batch** gradient descent, this is trivial, as it uses all points for computing the loss — **one epoch** is the same as **one update**. For **stochastic** gradient descent, **one epoch** means **N updates**, while for **mini-batch** (of size n), **one epoch** has **N/n updates**.*

---

Repeating this process over and over, for **many epochs**, is, in a nutshell, **training** a model.

### Exercise 1: put the previous pieces of code together and loop over many epochs

In [ ]:
# Defines number of epochs
n_epochs = 1000

...

In [ ]:
print(a, b)

Just keep in mind that, if you **don’t** use batch gradient descent (our example does),you’ll have to write an **inner loop** to perform the **five training steps** for either each **individual point** (**stochastic**) or **n points** (**mini-batch**). We’ll see a mini-batch example later down the line.

### Sanity Check

Just to make sure we haven’t done any mistakes in our code, we can use *Scikit-Learn’s Linear Regression* to fit the model and compare the coefficients.

In [ ]:
# Sanity Check: do we get the same results as our gradient descent?
from sklearn.linear_model import LinearRegression
linr = LinearRegression()
linr.fit(x_train, y_train)
print(linr.intercept_, linr.coef_[0])

They **match** up to 6 decimal places — we have a *fully working implementation of linear regression* using Numpy.

**Numpy?! Wait a minute… I thought this tutorial was about PyTorch!**

Yes, it is, but this served **two purposes**: *first*, to introduce the **structure** of our task, which will remain largely the same and, *second*, to show you the main **pain points** so you can fully appreciate how much PyTorch makes your life easier :-)

## PyTorch

In [ ]:
# !pip install --quiet torchviz
import torch
import torch.optim as optim
import torch.nn as nn
from torchviz import make_dot

First, we need to cover a **few basic concepts** that may throw you off-balance if you don’t grasp them well enough before going full-force on modeling.

In Deep Learning, we see **tensors** everywhere. Well, Google’s framework is called *TensorFlow* for a reason! *What is a tensor, anyway*?

### Tensors

In *Numpy*, you may have an **array** that has **three dimensions**, right? That is, technically speaking, a **tensor**.

A **scalar** (a single number) has **zero** dimensions, a **vector has one** dimension, a **matrix has two** dimensions and a **tensor has three or more dimensions**. That’s it!

But, to keep things simple, it is commonplace to call vectors and matrices tensors as well — so, from now on, **everything is either a scalar or a tensor**.

![alt text](http://karlstratos.com/drawings/linear_dogs.jpg)
Tensors are just higher-dimensional matrices :-) [Source](http://karlstratos.com)

You can create **tensors** in PyTorch pretty much the same way you create **arrays** in Numpy. Using [**tensor()**](https://pytorch.org/docs/stable/torch.html#torch.tensor) you can create either a scalar or a tensor.

PyTorch's tensors have equivalent functions as its Numpy counterparts, like: [**ones()**](https://pytorch.org/docs/stable/torch.html#torch.ones), [**zeros()**](https://pytorch.org/docs/stable/torch.html#torch.zeros), [**rand()**](https://pytorch.org/docs/stable/torch.html#torch.rand), [**randn()**](https://pytorch.org/docs/stable/torch.html#torch.randn) and many more.

In [ ]:
scalar = torch.tensor(3.14159)
vector = torch.tensor([1, 2, 3])
matrix = torch.ones((2, 3), dtype=torch.float)
tensor = torch.randn((2, 3, 4), dtype=torch.float)

print(scalar)
print(vector)
print(matrix)
print(tensor)

You can get the shape of a tensor using its [**size()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.size) method or its **shape** attribute.

In [ ]:
print(tensor.size(), tensor.shape)

You can also reshape a tensor using its [**reshape()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape) or [**view()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) methods.

Beware: these methods create a new tensor with the desired shape that **shares the underlying data** with the original tensor!

In [ ]:
new_tensor1 = tensor.reshape(2, -1)
new_tensor2 = tensor.view(2, -1)
print(new_tensor1.shape, new_tensor2.shape)

If you want to copy all data for real, that is, duplicate it in memory, you should use either its [**copy_()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.copy_) or [**clone()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.clone) methods.

### Loading Data, Devices and CUDA

”*How do we go from Numpy’s arrays to PyTorch’s tensors*”, you ask? 

That’s what [**from_numpy()**](https://pytorch.org/docs/stable/torch.html#torch.from_numpy) is good for. It returns a **CPU tensor**, though.

You can also easily **cast** it to a lower precision (32-bit float) using [**float()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.float).

In [ ]:
# Our data was in Numpy arrays, but we need to transform them into PyTorch's Tensors
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

print(type(x_train), type(x_train_tensor))

“*But I want to use my fancy GPU…*”, you say.

No worries, that’s what [**to()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.to) is good for. It sends your tensor to whatever **device** you specify, including your **GPU** (referred to as `cuda` or `cuda:0`).

In [ ]:
# Our data was in Numpy arrays, but we need to transform them into PyTorch's Tensors
x_train_tensor = torch.from_numpy(x_train).float().to('cuda')
y_train_tensor = torch.from_numpy(y_train).float().to('cuda')

print(type(x_train), type(x_train_tensor))

“*What if I want my code to fallback to CPU if no GPU is available?*”, you may be wondering… 

PyTorch got your back once more — you can use [**cuda.is_available()**](https://pytorch.org/docs/stable/cuda.html?highlight=is_available#torch.cuda.is_available) to find out if you have a GPU at your disposal and set your device accordingly.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Our data was in Numpy arrays, but we need to transform them into PyTorch's Tensors
x_train_tensor = torch.from_numpy(x_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)

print(type(x_train), type(x_train_tensor))

If you compare the **types** of both variables, you’ll get what you’d expect: `numpy.ndarray` for the first one and `torch.Tensor` for the second one.

But where does your nice tensor “live”? In your CPU or your GPU? You can’t say… but if you use PyTorch’s **type()**, it will reveal its **location** — `torch.cuda.FloatTensor` — a GPU tensor in this case.

In [ ]:
print(x_train_tensor.type())

We can also go the other way around, turning tensors back into Numpy arrays, using [**numpy()**](https://pytorch.org/docs/stable/tensors.html?highlight=numpy#torch.Tensor.numpy). It should be easy as `x_train_tensor.numpy()` but…

In [ ]:
x_train_tensor.numpy()

Unfortunately, Numpy **cannot** handle GPU tensors… you need to make them CPU tensors first using [**cpu()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.cpu).

In [ ]:
x_train_tensor.cpu().numpy()

### Creating Tensor for Parameters

What distinguishes a *tensor* used for *data* — like the ones we’ve just created — from a **tensor** used as a (*trainable*) **parameter/weight**?

The latter tensors require the **computation of its gradients**, so we can **update** their values (the parameters’ values, that is). That’s what the **`requires_grad=True`** argument is good for. It tells PyTorch we want it to compute gradients for us.

You may be tempted to create a simple tensor for a parameter and, later on, send it to your chosen device, as we did with our data, right? Let's see how it goes...

In [ ]:
# FIRST
# Initializes parameters "a" and "b" randomly, ALMOST as we did in Numpy
# since we want to apply gradient descent on these parameters, we need
# to set REQUIRES_GRAD = TRUE
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)
print(a, b)

The chunk of code above creates two nice tensors for our parameters, gradients and all. But they are **CPU** tensors.

What if we want **GPU** tensors? Let's try sending them to a `cuda` device...

In [ ]:
# SECOND
# But what if we want to run it on a GPU? We could just send them to device, right?
a = torch.randn(1, requires_grad=True, dtype=torch.float).to(device)
b = torch.randn(1, requires_grad=True, dtype=torch.float).to(device)
print(a, b)

In this second chunk of code, we tried the **naive** approach of sending them to our GPU. We succeeded in sending them to another device, but we **”lost”** the **gradients** somehow… **THIS ONLY HAPPENS IF DEVICE IS NOT CPU!**

Maybe we should **first** send our tensors to the **device** and only **then** use `requires_grad_()` method to set its `requires_grad` attribute to `True` in place.

---

*In PyTorch, every method that **ends** with an **underscore (_)** makes changes **in-place**, meaning, they will **modify** the underlying variable.*

---

In [ ]:
# THIRD
# We can either create regular tensors and send them to the device (as we did with our data)
a = torch.randn(1, dtype=torch.float).to(device)
b = torch.randn(1, dtype=torch.float).to(device)
# and THEN set them as requiring gradients...
a.requires_grad_()
b.requires_grad_()
print(a, b)

This approach worked just fine, but it is much better to **assign** tensors to a **device** at the moment of their **creation**.

In [ ]:
# We can specify the device at the moment of creation - RECOMMENDED!
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
print(a, b)

Much easier, right?

Now that we know how to create tensors that require gradients, let’s see how PyTorch handles them — that’s the role of the…

## Autograd

Autograd is PyTorch’s *automatic differentiation package*. Thanks to it, we **don’t need to worry** about partial derivatives, chain rule or anything like it.

### backward

So, how do we tell PyTorch to do its thing and **compute all gradients**? That’s what [**backward()**](https://pytorch.org/docs/stable/autograd.html#torch.autograd.backward) is good for.

Do you remember the **starting point** for **computing the gradients**? It was the **loss**, as we computed its partial derivatives w.r.t. our parameters. Hence, we need to invoke the `backward()` method from the corresponding Python variable, like, `loss.backward()`.

### Exercise 2: re-implement steps 1 to 3 using tensors and `backward()`

In [ ]:
# Step 1
# Computes our model's predicted output
yhat = ...

# Step 2    
# How wrong is our model? That's the error! 
error = ...
# It is a regression, so it computes mean squared error (MSE)
loss = ...

# Step 3    
...

### grad / zero_


What about the **actual values** of the **gradients**? We can inspect them by looking at the [**grad**](https://pytorch.org/docs/stable/autograd.html#torch.Tensor.grad) **attribute** of a tensor.

In [ ]:
print(a.grad, b.grad)

If you check the method’s documentation, it clearly states that **gradients are accumulated**. 

You can check this out by running the two code cells above again.

So, every time we use the **gradients** to **update** the parameters, we need to **zero the gradients afterwards**. And that’s what [**zero_()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.zero_) is good for.

In [ ]:
a.grad.zero_(), b.grad.zero_()

So, let’s **ditch** the **manual computation of gradients** and use both `backward()` and `zero_()` methods instead.

And, we are still missing **Step 4**, that is, **updating the parameters**. Let's include it as well...

### Exercise 3: update the parameters using `grad` attribute

In [ ]:
# Step 0
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# Step 1
# Computes our model's predicted output
yhat = a + b * x_train_tensor

# Step 2    
# How wrong is our model? That's the error! 
error = (y_train_tensor - yhat)
# It is a regression, so it computes mean squared error (MSE)
loss = (error ** 2).mean()

# Step 3    
# No more manual computation of gradients! 
loss.backward()
# Computes gradients for both "a" and "b" parameters
# a_grad = -2 * error.mean()
# b_grad = -2 * (x_train_tensor * error).mean()
print(a.grad, b.grad)

# Step 4
# Updates parameters using gradients and the learning rate
...

What happened?! In this first attempt, if we use the same update structure as in our *Numpy* code, we’ll get the weird **error** above but we can get a *hint* of what’s going on by looking at the tensor itself — once again we **“lost”** the **gradient** while reassigning the update results to our parameters. Thus, the `grad` attribute turns out to be `None` and it raises the error…

We can change it slightly, using a familiar **in-place Python assignment** as a second attempt. Try commenting the first update attempt, uncomment the other two lines of code and re-run it... did it work?

---

It turns out to be a case of ***“too much of a good thing”***. The culprit is PyTorch’s ability to build a **dynamic computation graph** from every **Python operation** that involves any **gradient-computing tensor** or its **dependencies**.

---

Don't worry, we’ll go deeper into the inner workings of the dynamic computation graph in the next section.

### no_grad

So, how do we tell PyTorch to “**back off**” and let us **update our parameters** without messing up with its **fancy dynamic computation graph**? That’s what `torch.no_grad()` is good for. It allows us to **perform regular Python operations on tensors, independent of PyTorch’s computation graph**.

In [ ]:
lr = 1e-1
n_epochs = 1000

# Step 0
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

for epoch in range(n_epochs):
    # Step 1
    # Computes our model's predicted output
    yhat = a + b * x_train_tensor

    # Step 2    
    # How wrong is our model? That's the error! 
    error = (y_train_tensor - yhat)
    # It is a regression, so it computes mean squared error (MSE)
    loss = (error ** 2).mean()

    # Step 3    
    # No more manual computation of gradients! 
    loss.backward()

    # Step 4
    # Updates parameters using gradients and the learning rate
    with torch.no_grad():
        a -= lr * a.grad
        b -= lr * b.grad

    # PyTorch is "clingy" to its computed gradients, we need to tell it to let it go...
    a.grad.zero_()
    b.grad.zero_()

print(a, b)

Finally, we managed to successfully run our model and get the **resulting parameters**. Surely enough, they **match** the ones we got in our *Numpy*-only implementation.

Let's take a look at the **loss** at the end of the training...

In [ ]:
loss

What if we wanted to have it as a *Numpy* array? I guess we could just use **numpy()** again, right? (and **cpu()** as well, since our *loss* is in the `cuda` device...

In [ ]:
loss.cpu().numpy()

What happened here? Unlike our *data tensors*, the **loss tensor** is actually computing gradients - and in order to use **numpy**, we need to [**detach()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.detach_) that tensor from the computation graph first:

In [ ]:
loss.detach().cpu().numpy()

This seems like **a lot of work**, there must be an easier way! And there is one indeed: we can use [**item()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.item), for **tensors with a single element** or [**tolist()**](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.tolist) otherwise.

In [ ]:
print(loss.item(), loss.tolist())

## Dynamic Computation Graph

---

“Unfortunately, no one can be told what the dynamic computation graph is. You have to see it for yourself.” 

Morpheus

---

Jokes aside, I want **you** to **see the graph for yourself** too!

The [PyTorchViz](https://github.com/szagoruyko/pytorchviz) package and its `make_dot(variable)` method allows us to easily visualize a graph associated with a given Python variable.

So, let’s stick with the **bare minimum**: two (gradient computing) **tensors** for our parameters, predictions, errors and loss.

In [ ]:
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

yhat = a + b * x_train_tensor
error = y_train_tensor - yhat
loss = (error ** 2).mean()

Now let's plot the **computation graph** for the **yhat** variable.

In [ ]:
make_dot(yhat)

Let’s take a closer look at its components:

* **blue boxes**: these correspond to the **tensors** we use as **parameters**, the ones we’re asking PyTorch to **compute gradients** for;

* **gray box**: a **Python operation** that involves a **gradient-computing tensor or its dependencies**;

* **green box**: the same as the gray box, except it is the **starting point for the computation** of gradients (assuming the `**backward()**` method is called from the **variable used to visualize** the graph)— they are computed from the **bottom-up** in a graph.

Now, take a closer look at the **green box**: there are **two arrows** pointing to it, since it is **adding up two variables**, `a` and `b*x`. Seems obvious, right?

Then, look at the **gray box** of the same graph: it is performing a **multiplication**, namely, `b*x`. But there is only **one arrow** pointing to it! The arrow comes from the **blue box** that corresponds to our parameter `b`.

Why don’t we have a box for our **data x**? The answer is: **we do not compute gradients for it**! So, even though there are *more* tensors involved in the operations performed by the computation graph, it **only** shows **gradient-computing tensors and its dependencies**.

Try using the `make_dot` method to plot the **computation graph** of other variables, like `error` or `loss`.

The **only difference** between them and the first one is the number of **intermediate steps (gray boxes)**.



In [ ]:
make_dot(loss)

What would happen to the computation graph if we set **`requires_grad`** to **`False`** for our parameter **`a`**?

In [ ]:
a_nograd = torch.randn(1, requires_grad=False, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

yhat = a_nograd + b * x_train_tensor

In [ ]:
make_dot(yhat)

Unsurprisingly, the **blue box** corresponding to the **parameter a** is no more! 

Simple enough: **no gradients, no graph**.

The **best thing** about the *dynamic computing graph* is the fact that you can make it **as complex as you want it**. You can even use *control flow statements* (e.g., if statements) to **control the flow of the gradients** (obviously!) :-)

Let's build a nonsensical, yet complex, computation graph just to make a point!

In [ ]:
yhat = a + b * x_train_tensor
error = y_train_tensor - yhat

loss = (error ** 2).mean()

if loss > 0:
    yhat2 = b * x_train_tensor
    error2 = y_train_tensor - yhat2

loss += error2.mean()

In [ ]:
make_dot(loss)

## Optimizer

So far, we’ve been **manually** updating the parameters using the computed gradients. That’s probably fine for *two parameters*… but what if we had a **whole lot of them**?! We use one of PyTorch’s **optimizers**, like [SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD) or [Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam).

In the code below, we create a *Stochastic Gradient Descent* (SGD) optimizer to update our parameters **a** and **b**.

---

Don’t be fooled by the **optimizer’s** name: if we use **all training data** at once for the update — as we are actually doing in the code — the optimizer is performing a **batch** gradient descent, despite of its name.

---

In [ ]:
# Our parameters
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

# Learning rate
lr = 1e-1

# Defines a SGD optimizer to update the parameters
optimizer = optim.SGD([a, b], lr=lr)

### step / zero_grad

An optimizer takes the **parameters** we want to update, the **learning rate** we want to use (and possibly many other hyper-parameters as well!) and **performs the updates** through its [**`step()`**](https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer.step) method.

Besides, we also don’t need to zero the gradients one by one anymore. We just invoke the optimizer’s [**`zero_grad()`**](https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer.zero_grad) method and that’s it!

### Exercise 4: Implement step 4 using the optimizer

In [ ]:
n_epochs = 1000

for epoch in range(n_epochs):
    # Step 1
    yhat = a + b * x_train_tensor

    # Step 2
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()

    # Step 3
    loss.backward()    
    
    # Step 4
    ...

print(a, b)

Cool! We’ve *optimized* the **optimization** process :-) What’s left?

## Loss

We now tackle the **loss computation**. As expected, PyTorch got us covered once again. There are many [loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions) to choose from, depending on the task at hand. Since ours is a regression, we are using the [Mean Square Error (MSE)](https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss) loss.

---

Notice that `nn.MSELoss` actually **creates a loss function** for us — **it is NOT the loss function itself**. Moreover, you can specify a **reduction method** to be applied, that is, **how do you want to aggregate the results for individual points** — you can average them (reduction=’mean’) or simply sum them up (reduction=’sum’).

---

In [ ]:
# Defines a MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

loss_fn

In [ ]:
fake_labels = torch.tensor([1., 2., 3.])
fake_preds = torch.tensor([1., 3., 5.])

loss_fn(fake_labels, fake_preds)

We then **use** the created loss function to compute the loss given our **predictions** and our **labels**.

### Exercise 5: implement step 2 using the loss function

In [ ]:
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

lr = 1e-1
n_epochs = 1000

# Defines a MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

optimizer = optim.SGD([a, b], lr=lr)

for epoch in range(n_epochs):
    # Step 1
    yhat = a + b * x_train_tensor
    
    # Step 2
    loss = ...

    # Step 3
    loss.backward() 

    # Step 4
    optimizer.step()
    optimizer.zero_grad()
    
print(a, b)

At this point, there’s only one piece of code left to change: the **predictions**. It is then time to introduce PyTorch’s way of implementing a…

## Model

In PyTorch, a **model** is represented by a regular **Python class** that inherits from the [**Module**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) class.

The most fundamental methods it needs to implement are:

* **\__init__(self)**: **it defines the parts that make up the model** —in our case, two parameters, **a** and **b**.

* **forward(self, x)**: it performs the **actual computation**, that is, it **outputs a prediction**, given the input **x**.

---

You are **not** limited to defining parameters, though… **models can contain other models (or layers) as its attributes** as well, so you can easily nest them. We’ll see an example of this shortly as well.

---

---

You should **NOT call the `forward(x)`** method, though. You should **call the whole model itself**, as in **`model(x)`** to perform a forward pass and output predictions.

---

Let’s build a proper (yet simple) model for our regression task. It should look like this:

In [ ]:
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # To make "a" and "b" real parameters of the model, we need to wrap them with nn.Parameter
        self.a = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        
    def forward(self, x):
        # Computes the outputs / predictions
        return self.a + self.b * x

### Parameter


In the **\__init__** method, we define our **two parameters**, **a** and **b**, using the [**Parameter()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Parameter) class, to tell PyTorch these **tensors should be considered parameters of the model they are an attribute of**.

Why should we care about that? By doing so, we can use our model’s [**parameters()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.parameters) method to retrieve **an iterator over all model’s parameters**, even those parameters of **nested models**, that we can use to feed our optimizer (instead of building a list of parameters ourselves!).

In [ ]:
dummy = ManualLinearRegression()

list(dummy.parameters())

Moreover, we can get the **current values for all parameters** using our model’s [**state_dict()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.state_dict) method.

In [ ]:
dummy.state_dict()

### state_dict

The **state_dict()** of a given model is simply a Python dictionary that **maps each layer / parameter to its corresponding tensor**. But only **learnable** parameters are included, as its purpose is to keep track of parameters that are going to be updated by the **optimizer**.

The **optimizer** itself also has a **state_dict()**, which contains its internal state, as well as the hyperparameters used.

It turns out **state_dicts** can also be used for **checkpointing** a model, as we will see later down the line.

In [ ]:
optimizer.state_dict()

### Device

**IMPORTANT**: we need to **send our model to the same device where the data is**. If our data is made of GPU tensors, our model must “live” inside the GPU as well.

In [ ]:
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Now we can create a model and send it at once to the device
model = ManualLinearRegression().to(device)
# We can also inspect its parameters using its state_dict
print(model.state_dict())

### train

In PyTorch, models have a [**train()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.train) method which, somewhat disappointingly, **does NOT perform a training step**. Its only purpose is to **set the model to training mode**. 

Why is this important? Some models may use mechanisms like [**Dropout**](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout), for instance, which have **distinct behaviors in training and evaluation phases**.

### Exercise 6: implement step 1 using the model (do not forget to set its mode)

In [ ]:
lr = 1e-1
n_epochs = 1000

loss_fn = nn.MSELoss(reduction='mean')
# Now the optimizers uses the parameters from the model
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    # Step 1
    yhat = ...
    
    # Step 2
    loss = loss_fn(yhat, y_train_tensor)
    # Step 3
    loss.backward()
    # Step 4
    optimizer.step()
    optimizer.zero_grad()
    
print(model.state_dict())

Now, the printed statements will look like this — final values for parameters **a** and **b** are still the same, so everything is ok :-)

### Nested Models

In our model, we manually created two parameters to perform a linear regression. Let’s use PyTorch’s [**Linear**](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear) layer as an attribute of our own, thus creating a nested model.

Even though this clearly is a contrived example, as we are pretty much wrapping the underlying model without adding anything useful (or, at all!) to it, it illustrates well the concept.

In the **\__init__** method, we created an attribute that contains our **nested `Linear` model**.

In the **forward()** method, we **call the nested model itself** to perform the forward pass (we should **not** call self.linear.forward(x)!).

### Exercise 7: set a Linear layer up for our regression model

In [ ]:
class LayerLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # Instead of our custom parameters, we use a Linear layer with single input and single output
        self.linear = ...
                
    def forward(self, x):
        # Now it only takes a call to the layer to make predictions
        return ...

Now, if we call the **parameters()** method of this model, **PyTorch will figure the parameters of its attributes in a recursive way**.

You can also add new `Linear` attributes and, even if you don’t use them at all in the forward pass, they will **still** be listed under `parameters()`.

In [ ]:
dummy = LayerLinearRegression()

list(dummy.parameters())

In [ ]:
dummy.state_dict()

### Layers

There are **MANY** different layers that can be uses in PyTorch:
- [Convolution Layers](https://pytorch.org/docs/stable/nn.html#convolution-layers)
- [Pooling Layers](https://pytorch.org/docs/stable/nn.html#pooling-layers)
- [Padding Layers](https://pytorch.org/docs/stable/nn.html#padding-layers)
- [Non-linear Activations](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)
- [Normalization Layers](https://pytorch.org/docs/stable/nn.html#normalization-layers)
- [Recurrent Layers](https://pytorch.org/docs/stable/nn.html#recurrent-layers)
- [Transformer Layers](https://pytorch.org/docs/stable/nn.html#transformer-layers)
- [Linear Layers](https://pytorch.org/docs/stable/nn.html#linear-layers)
- [Dropout Layers](https://pytorch.org/docs/stable/nn.html#dropout-layers)
- [Sparse Layers (embbedings)](https://pytorch.org/docs/stable/nn.html#sparse-layers)
- [Vision Layers](https://pytorch.org/docs/stable/nn.html#vision-layers)
- [DataParallel Layers (multi-GPU)](https://pytorch.org/docs/stable/nn.html#dataparallel-layers-multi-gpu-distributed)
- [Flatten Layer](https://pytorch.org/docs/stable/nn.html#flatten)

We have just used a **Linear** layer. Later on this tutorial, we will use some other layers, like **Convolution**, **Pooling** and **Non-linear Activations**.

### Sequential Models

Our model was simple enough… You may be thinking: “*why even bother to build a class for it?!*” Well, you have a point…

For **straightforward models**, that use **run-of-the-mill layers**, where the output of a layer is sequentially fed as an input to the next, we can use a, er… [**Sequential**](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) model :-)

In our case, we would build a Sequential model with a single argument, that is, the Linear layer we used to train our linear regression. The model would look like this:

In [ ]:
model = nn.Sequential(nn.Linear(1, 1)).to(device)

Simple enough, right?

### Training Step

So far, we’ve defined:
* an **optimizer**

* a **loss function**

* a **model**

Scroll up a bit and take a quick look at the code inside the loop. Would it **change** if we were using a **different optimizer**, or **loss**, or even **model**? If not, how can we make it more generic?

Well, I guess we could say all these lines of code **perform a training step**, given those **three elements** (optimizer, loss and model),the **features** and the **labels**.

So, how about **writing a function that takes those three elements** and **returns another function that performs a training step**, taking a set of features and labels as arguments and returning the corresponding loss?

In [ ]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Step 1: Makes predictions
        yhat = model(x)
        # Step 2: Computes loss
        loss = loss_fn(yhat, y)
        # Step 3: Computes gradients
        loss.backward()
        # Step 4: Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

Then we can use this general-purpose function to build a **train_step()** function to be called inside our training loop.

In [ ]:
lr = 1e-1

# Create a MODEL, a LOSS FUNCTION and an OPTIMIZER
# model = ManualLinearRegression().to(device)
# model = LayerLinearRegression().to(device)
model = nn.Sequential(nn.Linear(1, 1)).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)

train_step

Now our code should look like this… see how **tiny** the training loop is now?

In [ ]:
n_epochs = 1000

losses = []
# For each epoch...
for epoch in range(n_epochs):
    # Performs one train step and returns the corresponding loss
    loss = train_step(x_train_tensor, y_train_tensor)
    losses.append(loss)
    
# Checks model's parameters
print(model.state_dict())

In [ ]:
plt.plot(losses[:200])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')

Let’s give our training loop a rest and focus on our **data** for a while… so far, we’ve simply used our *Numpy arrays* turned **PyTorch tensors**. But we can do better, we can build a…

## Dataset

In PyTorch, a **dataset** is represented by a regular **Python class** that inherits from the [**Dataset**](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class. You can think of it as a kind of a Python **list of tuples**, each tuple corresponding to **one point (features, label)**.

The most fundamental methods it needs to implement are:

* **\__init__(self)**: it takes **whatever arguments** needed to build a **list of tuples** — it may be the name of a CSV file that will be loaded and processed; it may be two tensors, one for features, another one for labels; or anything else, depending on the task at hand.

* **\__get_item__(self, index)**: it allows the dataset to be **indexed**, so it can work like a list (`dataset[i]`) — it must **return a tuple (features, label)** corresponding to the requested data point. We can either return the **corresponding slices** of our **pre-loaded** dataset or tensors or, as mentioned above, **load them on demand** (like in this [example](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)).

* **\__len__(self)**: it should simply return the **size** of the whole dataset so, whenever it is sampled, its indexing is limited to the actual size.

---

There is **no need to load the whole dataset in the constructor method** (\__init__). If your **dataset is big** (tens of thousands of image files, for instance), loading it at once would not be memory efficient. It is recommended to **load them on demand** (whenever __get_item__ is called).

---

Let’s build a simple custom dataset that takes two tensors as arguments: one for the features, one for the labels. For any given index, our dataset class will return the corresponding slice of each of those tensors. It should look like this:

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

In [ ]:
# Wait, is this a CPU tensor now? Why? Where is .to(device)?
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

---

Did you notice we built our **training tensors** out of Numpy arrays but we **did not send them to a device**? So, they are **CPU** tensors now! **Why**?

We **don’t want our whole training data to be loaded into GPU tensors**, as we have been doing in our example so far, because **it takes up space** in our precious **graphics card’s RAM**.

---

### TensorDataset

Besides, you may be thinking “*why go through all this trouble to wrap a couple of tensors in a class?*”. And, once again, you do have a point… if a dataset is nothing else but a **couple of tensors**, we can use PyTorch’s [**TensorDataset**](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset) class, which will do pretty much what we did in our custom dataset above.

In [ ]:
from torch.utils.data import TensorDataset
train_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

OK, fine, but then again, **why** are we building a dataset anyway? We’re doing it because we want to use a…

## DataLoader

Until now, we have used the **whole training data** at every training step. It has been **batch gradient descent** all along. This is fine for our *ridiculously small dataset*, sure, but if we want to go serious about all this, we **must use mini-batch** gradient descent. Thus, we need mini-batches. Thus, we need to **slice** our dataset accordingly. 

Do you want to do it *manually*?! Me neither!

So we use PyTorch’s [**DataLoader**](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class for this job. We tell it which **dataset** to use (the one we just built in the previous section), the desired **mini-batch size** and if we’d like to **shuffle** it or not. That’s it!

Our **loader** will behave like an **iterator**, so we can **loop over it** and **fetch a different mini-batch** every time.

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

To retrieve a sample mini-batch, one can simply run the command below — it will return a list containing two tensors, one for the features, another one for the labels.

In [ ]:
next(iter(train_loader))

How does this change our training loop? Let’s check it out!

In [ ]:
lr = 1e-1

# Create a MODEL, a LOSS FUNCTION and an OPTIMIZER
model = nn.Sequential(nn.Linear(1, 1)).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)

n_epochs = 1000

losses = []

for epoch in range(n_epochs):
    # inner loop
    for x_batch, y_batch in train_loader:
        # the dataset "lives" in the CPU, so do our mini-batches
        # therefore, we need to send those mini-batches to the
        # device where the model "lives"
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        loss = train_step(x_batch, y_batch)
        losses.append(loss)
        
print(model.state_dict())

In [ ]:
plt.plot(losses)
plt.xlabel('Epochs (?)')
plt.ylabel('Loss')
plt.yscale('log')

Did you notice it is taking **longer** to train now? Can you guess **why**?

Two things are different now: not only we have an **inner loop** to load each and every **mini-batch** from our **DataLoader** but, more importantly, we are now **sending only one mini-batch to the device**.

---

For bigger datasets, **loading data sample by sample** (into a **CPU** tensor) using **Dataset’s \__get_item__** and then **sending all samples** that belong to the **same mini-batch at once to your GPU** (device) is the way to go in order to make the **best use of your graphics card’s RAM**.

Moreover, if you have **many GPUs** to train your model on, it is best to keep your dataset “agnostic” and assign the batches to different GPUs during training.

---

So far, we’ve focused on the **training data** only. We built a *dataset* and a *data loader* for it. We could do the same for the **validation** data, using the **split** we performed at the beginning of this post… or we could use **random_split** instead.

### random_split

PyTorch’s [**random_split()**](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) method is an easy and familiar way of performing a **training-validation split**. Just keep in mind that, in our example, we need to apply it to the **whole dataset** (not the *training* dataset we built in couple of sections ago).

Then, for each subset of data, we build a corresponding DataLoader, so our code looks like this:

In [ ]:
from torch.utils.data.dataset import random_split

# builds tensors from numpy arrays BEFORE split
x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

# builds dataset containing ALL data points
dataset = TensorDataset(x_tensor, y_tensor)

# performs the split
train_dataset, val_dataset = random_split(dataset, [80, 20])

# builds a loader of each set
train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

Now we have a **data loader** for our **validation** set, so, it makes sense to use it for the…

## Evaluation

Now, we need to change the training loop to include the **evaluation of our model**, that is, computing the **validation loss**. The first step is to include another inner loop to handle the *mini-batches* that come from the *validation loader* , sending them to the same *device* as our model. Next, we make **predictions** using our model and compute the corresponding **loss**.

That’s pretty much it, but there are **two small, yet important**, things to consider:

* [**torch.no_grad()**](https://pytorch.org/docs/stable/autograd.html#torch.autograd.no_grad): even though it won’t make a difference in our simple model, it is a **good practice to wrap the validation inner loop with this context manager to disable any gradient calculation** that you may inadvertently trigger — **gradients belong in training**, not in validation steps;
    
* [**eval()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.eval): the only thing it does is **setting the model to evaluation mode** (just like its `train()` counterpart did), so the model can adjust its behavior regarding some operations, like [**Dropout**](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout).

### Exercise 8: use the validation loader to compute the average loss over all batches

In [ ]:
torch.manual_seed(42)

# builds tensors from numpy arrays BEFORE split
x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

# builds dataset containing ALL data points
dataset = TensorDataset(x_tensor, y_tensor)

# performs the split
train_dataset, val_dataset = random_split(dataset, [80, 20])

# builds a loader of each set
train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

# defines learning rate
lr = 1e-1

# Create a MODEL, a LOSS FUNCTION and an OPTIMIZER
model = nn.Sequential(nn.Linear(1, 1)).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)

n_epochs = 1000

losses = []
val_losses = []

# Looping through epochs...
for epoch in range(n_epochs):
    # TRAINING
    batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        loss = train_step(x_batch, y_batch)
        batch_losses.append(loss)

    losses.append(np.mean(batch_losses))
        
    # VALIDATION
    # no gradients in validation!
    with torch.no_grad():
        val_batch_losses = []

        ...
        
        val_losses.append(np.mean(val_batch_losses))

print(model.state_dict())

### Training Loop

The training loop should be a stable structure, so we can organize it into functions as well...
Let's build a function for **validation** and another one for the **training loop** itself, training step and all!

In [ ]:
def validation(model, loss_fn, val_loader):
    # Figures device from where the model parameters (hence, the model) are
    device = next(model.parameters()).device.type
    val_losses = []

    # no gradients in validation!
    with torch.no_grad():
        val_batch_losses = []
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            # sets model to EVAL mode
            model.eval()

            # make predictions
            yhat = model(x_val)
            val_loss = loss_fn(yhat, y_val)
            val_batch_losses.append(val_loss.item())

        val_losses.append(np.mean(val_batch_losses))

    return val_losses


def train_loop(model, loss_fn, optimizer, n_epochs, train_loader, val_loader=None):
    # Figures device from where the model parameters (hence, the model) are
    device = next(model.parameters()).device.type
    # Creates the train_step function for our model, loss function and optimizer
    train_step = make_train_step(model, loss_fn, optimizer)

    losses = []
    val_losses = []

    for epoch in range(n_epochs):
        # TRAINING
        batch_losses = []
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            loss = train_step(x_batch, y_batch)
            batch_losses.append(loss)

        losses.append(np.mean(batch_losses))

        # VALIDATION
        if val_loader is not None:
            val_loss = validation(model, loss_fn, val_loader)
            val_losses.append(val_loss)

        print("Epoch {} complete...".format(epoch))

    return losses, val_losses

In [ ]:
torch.manual_seed(42)

# builds tensors from numpy arrays BEFORE split
x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

# builds dataset containing ALL data points
dataset = TensorDataset(x_tensor, y_tensor)

# performs the split
train_dataset, val_dataset = random_split(dataset, [80, 20])

# builds a loader of each set
train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

# defines learning rate
lr = 1e-1

# Create a MODEL, a LOSS FUNCTION and an OPTIMIZER
model = nn.Sequential(nn.Linear(1, 1)).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

n_epochs = 1000

losses, val_losses = train_loop(model, loss_fn, optimizer, n_epochs, train_loader, val_loader)

print(model.state_dict())

In [ ]:
plt.plot(losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

"*Wait, there is something weird with this plot...*", you say. You're right, the **validation loss** is **smaller** than the **training loss**. Shouldn't it be the other way around?! Well, generally speaking, *YES*, it should... but you can learn more about situations where this *swap* happens at this great [post](pyimg.co/kku35).

## TorchVision

[Torchvision](https://pytorch.org/docs/stable/torchvision/index.html) is a package containing popular datasets, model architectures and common image transformations for computer vision.

### Datasets

All [datasets](https://pytorch.org/docs/stable/torchvision/datasets.html) included in torchvision are subclasses of the [**Dataset**](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class we've seen earlier in this tutorial, so we can stick with the [**DataLoader**](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) as well.

We'll work with the canonical dataset for computer vision tutorials: [**MNIST**](https://pytorch.org/docs/stable/torchvision/datasets.html#mnist).

**Training Set**

http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz

http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz

**Test Set**

http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz

http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


In [ ]:
from torchvision import datasets

mnist_train = datasets.MNIST('./mnist', train=True, download=True)
mnist_test = datasets.MNIST('./mnist', train=False, download=True)

In [ ]:
mnist_train

In [ ]:
mnist_train.data.shape

In [ ]:
mnist_train.targets

There are 60k 28x28 images and the targets are the corresponding digits.

Let's take a look at the first sample, which is a **5**. You'll notice that, somewhat unsurprisingly, the sample is a **tensor**.

In [ ]:
sample_tensor = mnist_train.data[0]
plt.imshow(sample_tensor)
print(sample_tensor.type())

### Transforms

Torchvision has some common image transformations on its [**transforms**](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision-transforms) module. It is important to realize there are two main groups of transformations:

- transformations based on [**PIL images**](https://pytorch.org/docs/stable/torchvision/transforms.html#transforms-on-pil-image)
- transformations based on [**Tensors**](https://pytorch.org/docs/stable/torchvision/transforms.html#transforms-on-torch-tensor)

Obviously, there are transformations to convert from tensors [**ToPILImage**](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ToPILImage) and from PIL image [**ToTensor**](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ToTensor).

Let's try converting our sample tensor into a sample (PIL) image.

In [ ]:
from torchvision.transforms import ToPILImage

sample_img = ToPILImage()(sample_tensor)
plt.imshow(sample_img)

### Transforms on PIL Image

These transforms include the typical things you'd like to do with an image: [Resize](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Resize), [CenterCrop](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.CenterCrop), [GrayScale](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Grayscale), [RandomHorizontalFlip](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomHorizontalFlip), to name a few.

They take a **PIL Image** as inputs, not tensors. So, let's use our sample image from the previous step and try some **random horizontal flipping**. But, just to make sure we flip, let's ditch the randomness and make it flip a 100% of times.

In [ ]:
from torchvision.transforms import RandomHorizontalFlip

flipper = RandomHorizontalFlip(p=1.0)

In [ ]:
flipped_img = flipper(sample_img)
plt.imshow(flipped_img)

Ok, we have a **flipped 5** now.

Let's take a look at the other group of transformations now...

### Transforms on Tensor

These are only three transforms that take tensors as inputs: [LinearTransformation](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.LinearTransformation), [Normalize](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Normalize) and [RandomErasing](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomErasing) (although I believe this one was a better fit for the other group of transforms...).

Let's apply a **Normalize** transform to our **flipped 5**. But, in order to be able to do that, we need to make it back into a tensor using **ToTensor**.

In [ ]:
from torchvision.transforms import ToTensor

tensorizer = ToTensor()
img_tensor = tensorizer(flipped_img)
img_tensor

In [ ]:
img_tensor.min(), img_tensor.max()

### Normalize Transform

From our image tensor, we see its values are in the **[0, 1]** range.

Usually, when dealing with neural networks, it is better to have our inputs in a symmetrical range, like **[-1, 1]**.

And that's why we are going to use the **Normalize** transform. From PyTorch's extensive documentation, we get:

`Normalize a tensor image with mean and standard deviation.`

`Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input torch.*Tensor i.e.`

`input[channel] = (input[channel] - mean[channel]) / std[channel]`

So, if we would like to map our [0, 1] range into [-1, 1], we can set our **`mean` to 0.5** and our **`std` to 0.5** as well.

This way, we get:
- 0 input is transformed into (0 - .5)/.5 = -1
- 1 input is transformed into (1 - .5)/.5 = 1

---

Even though the transform is called **Normalize**, what we have just done with the inputs is actually a **min-max scaling**. We have **NOT** computed the **true mean** and **true standard deviation**, we have just made them equals 0.5 for our convenience.

Had we use the true values for both mean and standard deviation, we would have achieved a **standardization**, that is, our data would have **zero mean** and **unit standard deviation**.

---

Our images have only **1 channel**, so we only need to specify one element in each tuple.

In [ ]:
from torchvision.transforms import Normalize

normalizer = Normalize(mean=(.5,), std=(.5,))
normalized_tensor = normalizer(img_tensor)
normalized_tensor

In [ ]:
normalized_tensor.min(), normalized_tensor.max()

The range is [-1, 1] now.

**Mission accomplished!**

### Compose

Sure enough, we don't need to do transformations one by one: we can use [**Compose**](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Compose) for that.

This is straightforward: line all desired transformations up in a list. This is pretty much the same as a Pipeline in Scikit-Learn.

In [ ]:
from torchvision.transforms import Compose

composer = Compose([ToPILImage(),
                    RandomHorizontalFlip(p=1.0),
                    ToTensor(),
                    Normalize(mean=(.5,), std=(.5,))])

composed_tensor = composer(sample_tensor)

The resulting tensor should be exactly the same as the tensor we have transformed step-by-step... let's confirm that:

In [ ]:
(composed_tensor == normalized_tensor).all()

### Transforming Datasets

The best thing about composing transforms is the fact you can apply them whenever a given data point is being sampled from the dataset.

Remember the [**Dataset**](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class, which torchvision datasets use as parent class? It is possible to call a composed transformation, just like we did, inside the **\__get_item__(self, index)** method.

And that's exactly what torchvision datasets do! You can specify a **transform** argument and, whenever your [**DataLoader**](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) fetches some samples, they will be transformed already! **Beautiful**, uh?

In [ ]:
new_composer = Compose([RandomHorizontalFlip(p=.5),
                        ToTensor(),
                        Normalize(mean=(.5,), std=(.5,))])

mnist_train = datasets.MNIST('./mnist', train=True, download=True, transform=new_composer)
mnist_test = datasets.MNIST('./mnist', train=False, download=True, transform=new_composer)

train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True)
val_loader = DataLoader(mnist_test, batch_size=128, shuffle=True)

In [ ]:
transformed_sample = next(iter(train_loader))[0][0]
transformed_sample

In [ ]:
transformed_sample.min(), transformed_sample.max()

Just as expected!

## A Simple Neural Network

We will use the **MNIST** dataset and build a simple neural network to try classify the hand-written digits.

Besides, let's keep it simple and build a **Sequential** model, just like we did with our linear regression.

Our input images have 28x28 pixels, that is, 784 pixels in total. Our targets are 10 different classes (digits 0 to 9). So, our model can be structured as follows:

- **input layer**: 784 units
- **hidden layer(s) and non-linear activations**: we can get creative :-)
- **output layer**: 10 units

For now, let's use **one hidden layer** with **50 units** and let's use a [**ReLU**](https://pytorch.org/docs/stable/nn.html#relu) as non-linear activation.

Since our inputs are 2-dimensional (28x28) and our model has 784 inputs, we still need to add a [**Flatten**](https://pytorch.org/docs/stable/nn.html#flatten) layer at the beginning.

So, our model would look like this - let's use [**add_module**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.add_module) to **name** our layers:

In [ ]:
model = nn.Sequential()
model.add_module('flatten', nn.Flatten())
model.add_module('linear1', nn.Linear(784, 50))
model.add_module('relu1', nn.ReLU())
model.add_module('output', nn.Linear(50, 10))
model.to(device)

print(model)

If we use model's [**named_modules()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.named_modules) method, we can retrieve a list of all modules and the model itself!

In [ ]:
list(model.named_modules())

And we can use the layers / modules names using dot notation:

In [ ]:
model.linear1

We can also get its corresponding **weights**: `model.linear1.weight`

Let's take a look at the weights, plotting them:

In [ ]:
weights = model.linear1.weight
plt.hist(weights.detach().cpu().numpy().reshape(-1,))

In [ ]:
print(weights.min(), weights.max())

The linear layer has **39200 weights** and we can see they are **uniformly distributed in [-.0357, .0357] range**. Do you think this is a **good initialization scheme**?

### init

Initialization schemes are a big deal! It goes beyond the scope of this tutorial to go into more details, but we'll see how we can **initialize the weights** of a given layer in our neural network.

For more details on **weight initialization schemes**, you can check my [post](https://towardsdatascience.com/hyper-parameters-in-action-part-ii-weight-initializers-35aee1a28404) out :-)

PyTorch has an [**init**](https://pytorch.org/docs/stable/nn.init.html#torch-nn-init) module that has the most common initialization schemes, such as [**kaiming_uniform**](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_uniform_) and [**kaiming_normal**](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_normal_) - also known as **He initialization** - the recommended scheme for using with **ReLU** activation function.

Let's use **kaiming_normal_** (notice the **_** - we are making changes **in place**!) to initialize the weights of our linear layer. And let's zero our biases too.

In [ ]:
nn.init.kaiming_normal_(model.linear1.weight, mode='fan_in', nonlinearity='relu')
nn.init.constant_(model.linear1.bias, 0)

Did it work? Let's plot it!

In [ ]:
weights = model.linear1.weight
plt.hist(weights.detach().cpu().numpy().reshape(-1,))

Awesome! We have succesfully initialized the weights of our network!

### Softmax Layer for Classification

You may be thinking: "*this is a **multi-class classification** problem... where is the **softmax** layer at the end?*"

Well, you do have a point... but PyTorch makes things a bit confusing here... 

You **MAY** add a softmax layer at the end or, better yet, a [**LogSoftmax**](https://pytorch.org/docs/stable/nn.html#logsoftmax). If you **DO add it**, you **MUST** use the corresponding negative log-likelihood loss ([**NLLLoss**](https://pytorch.org/docs/stable/nn.html#nllloss)).

**BUT**

If you **DO NOT** add a **LogSoftmax** layer at the end, you **MUST** use the [**CrossEntropyLoss**](https://pytorch.org/docs/stable/nn.html#crossentropyloss) which, by itself, combines **LogSoftmax AND NLLLoss**.

---

In summary, you have two options:
- **do** add **LogSoftmax** layer at the end and use **NLLLoss**, or
- just use **CrossEntropyLoss**

---

We use the latter, as it is simpler.

### Training Step

Remember the **training step** function we built earlier? Its arguments were:
- a **model**: we have one, our neural network
- a **loss function**: our problem is a classification and we have not added a softmax layer, so we must use **CrossEntropyLoss**
- an **optimizer**: let's stick with **SGD**


In [ ]:
lr = 1e-1

loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)

### Training Loop

We can also reuse the code from the **Evaluation** section to perform the model training - using 10 instead of 1000 epochs.

In [ ]:
torch.manual_seed(42)

n_epochs = 10

losses, val_losses = train_loop(model, loss_fn, optimizer, n_epochs, train_loader, val_loader)

In [ ]:
plt.plot(losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

---

Isn't it **great**? We loaded the **MNIST dataset**, built a **neural network** model, defined the corresponding **loss function** and **reused everything else**.

---

What's left to do? We need to check our model's **accuracy** on the validation dataset. So, we make small changes to the **validation loop**, checking which class got the highest probability and matching it against our targets.

We can use [**torch.max()**](https://pytorch.org/docs/stable/torch.html#torch.max) with `dim=1` to get both the maximum value (highest predicted probability) and its index (which corresponds to the predicted digit).

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for x_val, y_val in val_loader:
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        
        model.eval()
        yhat = model(x_val)
        
        # this is PyTorch's version of argmax, but it returns a tuple: (max value, index of max value)
        _, predicted = torch.max(yhat, 1)
        # we get the size of the batch and add up to the total number of samples
        total += y_val.size(0)
        # we add how many samples got classified correctly
        correct += (predicted == y_val).sum().item()
        
print(correct/total)

**We got almost 95% accuracy**!

But, can we do better?

## A Convolutional Neural Network (CNN)

It is time to try something a bit more sophisticated!

Let's create our own model, inheriting from the [**Module**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) class. It will take one argument, the **number of features** we will create output in our convolution layer.

We will use [**Conv2d**](https://pytorch.org/docs/stable/nn.html#conv2d) and [**MaxPool2d**](https://pytorch.org/docs/stable/nn.html#maxpool2d) layers, apart from the already familiar **Linear**, **ReLU** and **Flatten**.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, n_feature):
        super(CNN, self).__init__()
        self.n_feature = n_feature
        # Creates the convolution layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=n_feature, out_channels=n_feature, kernel_size=5)
        # Creates the linear layers
        self.fc1 = nn.Linear(n_feature * 4 * 4, 50) # where do this 4 * 4 come from?! check it below
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x, verbose=False):
        # Input dimension: (1, 28, 28)
        # Output dimension: (n_feature, 24, 24) - we loose (5-1) pixels in each dimension due to the kernel and no padding
        x = self.conv1(x)
        x = F.relu(x)

        # Input dimension (n_feature, 24, 24)
        # Output dimension (n_feature, 12, 12)
        x = F.max_pool2d(x, kernel_size=2)

        # Input dimension (n_feature, 12, 12)
        # Output dimension (n_feature, 8, 8)
        x = self.conv2(x)
        x = F.relu(x)

        # Input dimension (n_feature, 8, 8)
        # Output dimension (n_feature, 4, 4) - we loose (5-1) pixels in each dimension due to the kernel and no padding
        x = F.max_pool2d(x, kernel_size=2)

        # Input dimension (n_feature, 4, 4)
        # Output dimension (n_feature * 4 * 4)
        x = nn.Flatten()(x)

        # Input dimension (n_feature * 4 * 4)
        # Output dimension (50)
        x = self.fc1(x)
        x = F.relu(x)
        
        # Input dimension (50)
        # Output dimension (10)
        x = self.fc2(x)
        return x

### Functional

Maybe you noticed already, but our CNN model used a **mix** of **layers** from both **torch.nn** and **torch.nn.functional**.

"*What is the difference?*", you ask?

PyTorch's [**functional**](https://pytorch.org/docs/stable/nn.functional.html#torch-nn-functional) layers are, er... **functions**.

Let's consider two different layers: **2d convolution** and **2d max pooling**. Both can be used as **layer** or **function**:
- [**Conv2d**](https://pytorch.org/docs/stable/nn.html#conv2d) or [**F.conv2d**](https://pytorch.org/docs/stable/nn.functional.html#conv2d)
- [**MaxPool2d**](https://pytorch.org/docs/stable/nn.html#maxpool2d) or [**F.max_pool2d**](https://pytorch.org/docs/stable/nn.functional.html#max-pool2d)

Why did we use a **Conv2d layer** but a **F.max_pool2d function**?

Well, on one hand, it is more **convenient** to use `F.max_pool2d(x, kernel_size=2)` instead of `nn.MaxPool2d(kernel_size=2)(x)`. It just looks better, I think...

But the thing is, **max pooling layers** do not have **parameters/weights** to be learned! **Convolution layers, do!**

Remember the **Nested Model** section? Every **model** or **layer** gets its **parameters** recursively accessed by the parent **model**, **provided they are attributes in the `__init__` method**!

---

So, it is pretty simple: does the layer has **parameters** that need to be learned?
- **YES** - Use **layers** from **torch.nn** and make them **attributes of the parent model**
- **NO** - feel free to use **functions** from **torch.nn.functional**

---

### Training Step

### Exercise 9: set the components up to make a train step

In [ ]:
lr = 1e-1

model = ...
loss_fn = ...
optimizer = ...

# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)

In [ ]:
print(model)

### Training Loop

You know the drill :-)

In [ ]:
torch.manual_seed(42)

n_epochs = 10

losses, val_losses = train_loop(model, loss_fn, optimizer, n_epochs, train_loader, val_loader)

In [ ]:
plt.plot(losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for x_val, y_val in val_loader:
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        
        model.eval()
        yhat = model(x_val)
        
        # this is PyTorch's version of argmax, but it returns a tuple: (max value, index of max value)
        _, predicted = torch.max(yhat, 1)
        # we get the size of the batch and add up to the total number of samples
        total += y_val.size(0)
        # we add how many samples got classified correctly
        correct += (predicted == y_val).sum().item()
        
print(correct/total)

**Awesome! We are close to 98% accuracy now!**

### Visualizing Filters

One of the cool things about Convolution Neural Networks is that its **features** or **filters** can be visualized, so we can have a glimpse of **what the network is looking for** inside the input images :-)

We can fetch the **weights** of a convolution layer...

In [ ]:
model.conv1.weight

This tensor is 6 x 5 x 5... why is that? It has **6 features or filters** (we configured it that way) and each one is **5 x 5** because that's the **kernel size** we defined. No surprises here!

But we can think of each one of this 5 x 5 tensors as **an image**!

In [ ]:
# Code adapted from "How to Visualize Filters and Feature Maps in Convolutional Neural Networks"
# by Jason Brownlee
# https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/

layer_name = 'conv1'
# retrieve weights from the hidden layer
filters = getattr(model, layer_name).weight.data.cpu().numpy()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

n_filters, ix = filters.shape[0], 1

for i in range(n_filters):
    # get the filter
    f = filters[i, :, :, :]
    # plot each channel separately
    for j in range(filters.shape[1]):
        # specify subplot and turn of axis
        ax = plt.subplot(n_filters, filters.shape[1], ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(f[j, :, :], cmap='gray')
        ix += 1

# show the figure
plt.show()

## Transfer Learning

"Transfer learning is a machine learning method where a model developed for a task is **reused** as the starting point for a model on a second task.

It is a popular approach in deep learning where pre-trained models are used as the starting point on computer vision and natural language processing tasks given the vast compute and time resources required to develop neural network models on these problems and from the huge jumps in skill that they provide on related problems."

Source: [A Gentle Introduction to Transfer Learning for Deep Learning](https://machinelearningmastery.com/transfer-learning-for-deep-learning/)

So, let's try using a **pre-trained model** as starting point for classifying MNIST's digits!

Let's take one the most **famous** neural network architectures for a spin...

### AlexNet

"AlexNet was the winning entry in ILSVRC 2012. It solves the problem of image classification where the input is an image of one of 1000 different classes (e.g. cats, dogs etc.) and the output is a vector of 1000 numbers."

To learn more about AlexNet, check out the post where this excerpt of text comes from: [Understanding AlexNet](https://www.learnopencv.com/understanding-alexnet/).

![alt text](https://www.learnopencv.com/wp-content/uploads/2018/05/AlexNet-1.png)

Source: [Learn OpenCV](https://www.learnopencv.com/understanding-alexnet/)

It is actually a bit **too much** to use AlexNet for the purpose of classifying MNIST's digits... but let's go for it anyway!

How do we use a **pre-trained model**? We first need to **download its weights** from somewhere and then **load them into the corresponding empty model architecture**.

To be honest, PyTorch provides a direct way of directly downloading the weights of AlexNet... but let's make it the **hard way**, downloading it manually and only then **loading them into the model**, so we learn how to actually load it.

### Loading Models

Let's start by downloading the pre-trained weights... AlexNet has about 230 Mb!

In [ ]:
!curl https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth --output alexnet-owt-4df8aa71.pth --verbose

Now, let's create the **empty** network architecture... we could build it from scratch based on the figure above, but it seems too much of a hassle - let's use TorchVision's [**models**](https://pytorch.org/docs/stable/torchvision/models.html#torchvision-models) module.

In [ ]:
from torchvision.models import alexnet

alex = alexnet()

print(alex)

First, we load the file into a **state_dict** (remember those?!) using [**torch.load()**](https://pytorch.org/docs/stable/torch.html#torch.load):

In [ ]:
state_dict = torch.load('alexnet-owt-4df8aa71.pth')
print(state_dict.keys())

Then we load the **state_dict** into the **empty architecture** we created. For this step, we use, obviously, the model's [**load_state_dict()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.load_state_dict) method.

In [ ]:
alex.load_state_dict(state_dict)

### Freezing Layers

OK, we have AlexNet locked and loaded! Should we just **start training it?** 

Well, not so fast... it has 60 million parameters! Besides, **what would be the point of transfer learning then**?

So, we are **freezing its layers**, meaning, we **do not want to update its weights**. How do we do that? **Turning gradients OFF!**

In [ ]:
for parameter in alex.parameters():
    parameter.requires_grad = False


We also need to make a small change... AlexNet was trained to classify **1000 classes**, but we have **only 10 digits**, so we need to **replace the last layer**.

In [ ]:
print(alex.classifier)

In [ ]:
alex.classifier[6] = nn.Linear(4096, 10)

Let's double check if the **only trainable parameters** are the ones from our newly created last layer: **classifier.6**.

For this, the model's [**named_parameters()**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.named_parameters) method come in handy:

In [ ]:
for name,param in alex.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

### Transforming Datasets (again)

AlexNet also expects **different inputs**: 3-channel 224 x 224 images.
But our **MNIST** data has single-channel 28 x 28 images. So, we can resort to **transforms** once again to transform the latter into the former.

In [ ]:
from torchvision.transforms import Resize, Grayscale, ToTensor, Normalize

alex_transforms = Compose([Resize(224),
                           Grayscale(num_output_channels=3),
                           ToTensor(),
                           Normalize((.5, .5, .5), (.5, .5, .5)),])

mnist_train = datasets.MNIST('./mnist', train=True, download=True, transform=alex_transforms)
mnist_test = datasets.MNIST('./mnist', train=False, download=True, transform=alex_transforms)

train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True)
val_loader = DataLoader(mnist_test, batch_size=128, shuffle=True)

### Training Step

Don't forget to **send Alex to the device** :-)

In [ ]:
lr = 1e-1

model = alex.to(device)
loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)

### Training Loop

Nothing new here, but Alex is big, so let's start with 2 epochs only...

In [ ]:
torch.manual_seed(42)

n_epochs = 2

losses, val_losses = train_loop(model, loss_fn, optimizer, n_epochs, train_loader, val_loader)

In [ ]:
plt.plot(losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for x_val, y_val in val_loader:
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        
        model.eval()
        yhat = model(x_val)
        
        # this is PyTorch's version of argmax, but it returns a tuple: (max value, index of max value)
        _, predicted = torch.max(yhat, 1)
        # we get the size of the batch and add up to the total number of samples
        total += y_val.size(0)
        # we add how many samples got classified correctly
        correct += (predicted == y_val).sum().item()
        
print(correct/total)

Doesn't look so impressive now, does it? Anyway...

### Saving Models

As you can see, a **bigger** model like AlexNet takes **quite some time for training each epoch**, even with all but one of its layers **frozen**.

So, it is important to be able to **checkpoint** our model, in case we'd like to **restart training later** (if you get disconnected from Google Colab, for instance!).

To checkpoint a model, we basically have to **save its state** into a file, to **load** it back later - nothing special, actually.

What defines the **state of a model**?
- **model.state_dict()**: kinda obvious, right?
- **optimizer.state_dict()**: remember optimizers had the state_dict as well?
- **loss**: after all, you should keep track of its evolution
- **epoch**: it is just a number, so why not? :-)
- **anything else you'd like to have restored**

Then, **wrap everything into a Python dictionary** and use [**torch.save()**](https://pytorch.org/docs/stable/torch.html?highlight=save#torch.save) to dump it all into a file! Easy peasy!

In [ ]:
checkpoint = {'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': losses,
              'val_loss': val_losses}

torch.save(checkpoint, 'alexnet_checkpoint.pth')

In [ ]:
!ls -l alexnet_checkpoint.pth

How would you **load** it back?

In [ ]:
checkpoint = torch.load('alexnet_checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
losses = checkpoint['loss']
val_losses = checkpoint['val_loss']

In [ ]:
plt.plot(losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

Seems about right...

You may save a model for **checkpointing**, like we have just done, or for **making predictions**, assuming training is finished.

After loading the model, **DO NOT FORGET**:

---

**SET THE MODE** (not the mood!):
- **checkpointing: model.train()**
- **predicting: model.eval()**

---

## Further Improvements


Is there **anything else** we can improve or change? Sure, there is **always something else** to add to your model — using a [**learning rate scheduler**](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate) or a [**learning rate finder**](https://github.com/davidtvs/pytorch-lr-finder).

But this tutorial is already waaaaay too long, so I will stop right here.

## Final Thoughts

I believe this tutorial has **most of the necessary steps** one needs go to trough in order to **learn**, in a **structured** and **incremental** way, how to **develop Deep Learning models using PyTorch**.

Hopefully, after finishing working through all code in this post, you’ll be able to better appreciate and more easily work your way through PyTorch’s official [tutorials](https://pytorch.org/tutorials/).

If you have any thoughts, comments or questions, please leave a comment below or contact me on [LinkedIn](https://br.linkedin.com/in/dvgodoy) or [Twitter](https://twitter.com/dvgodoy).